In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('cruise').getOrCreate()
print('Spark Session is Created')

Spark Session is Created


In [3]:
data = spark.read.format('libsvm').load('sample_libsvm_data.txt')
data.show()

24/01/10 16:47:28 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [4]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression()

# to split into train test set

train_data,test_data = data.randomSplit([0.75,0.25])

fit_model = lr.fit(train_data)

pred = fit_model.evaluate(test_data)

pred.predictions.show()




/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[100,101,102...|[1.02137427397438...|[0.73524020492069...|       0.0|
|  0.0|(692,[127,128,129...|[18.8694894445085...|[0.99999999361611...|       0.0|
|  0.0|(692,[152,153,154...|[10.9644821330736...|[0.99998269472930...|       0.0|
|  1.0|(692,[99,100,101,...|[-4.1101838565124...|[0.01613998556796...|       1.0|
|  1.0|(692,[119,120,121...|[3.36474654244989...|[0.96658442516164...|       0.0|
|  1.0|(692,[124,125,126...|[-23.097917050224...|[9.30469370791390...|       1.0|
|  1.0|(692,[124,125,126...|[-18.729380871393...|[7.34400958312227...|       1.0|
|  1.0|(692,[125,126,153...|[-20.995309937900...|[7.61820662750300...|       1.0|
|  1.0|(692,[126,127,128...|[-22.614705859117...|[1.50854418620376...|       1.0|
|  1.0|(692,[126

In [8]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator,MulticlassClassificationEvaluator
b_eval = BinaryClassificationEvaluator()
#m_eval = MulticlassClassificationEvaluator
b_eval.evaluate(pred.predictions)
#m_eval.evaluate(pred.predictions)

0.9743589743589743

In [30]:
# implementing titanic dataset into spark

df = spark.read.csv('titanic.csv',inferSchema=True,header=True)
df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| NULL|       S|
|          6|       0|     3|    Moran, Mr. James|  male|NULL|    0|    0|      

In [31]:
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [32]:
df.columns


['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [33]:
# to store column in the mycol

mycol = df.select(['Survived','Pclass','Sex','Age','SibSp','Parch','Fare','Embarked'])


In [34]:
myfinal = mycol.na.drop()
myfinal.show()

+--------+------+------+----+-----+-----+-------+--------+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|
+--------+------+------+----+-----+-----+-------+--------+
|       0|     3|  male|22.0|    1|    0|   7.25|       S|
|       1|     1|female|38.0|    1|    0|71.2833|       C|
|       1|     3|female|26.0|    0|    0|  7.925|       S|
|       1|     1|female|35.0|    1|    0|   53.1|       S|
|       0|     3|  male|35.0|    0|    0|   8.05|       S|
|       0|     1|  male|54.0|    0|    0|51.8625|       S|
|       0|     3|  male| 2.0|    3|    1| 21.075|       S|
|       1|     3|female|27.0|    0|    2|11.1333|       S|
|       1|     2|female|14.0|    1|    0|30.0708|       C|
|       1|     3|female| 4.0|    1|    1|   16.7|       S|
|       1|     1|female|58.0|    0|    0|  26.55|       S|
|       0|     3|  male|20.0|    0|    0|   8.05|       S|
|       0|     3|  male|39.0|    1|    5| 31.275|       S|
|       0|     3|female|14.0|    0|    0| 7.8542|       

In [37]:
from pyspark.ml.feature import VectorAssembler,VectorIndexer,StringIndexer,OneHotEncoder

gender_indexer = StringIndexer(inputCol='Sex',outputCol='SexIndexer')
gender_encoder = OneHotEncoder(inputCol='SexIndexer',outputCol='SexVec')


embark_indexer = StringIndexer(inputCol='Embarked',outputCol='EmbarkedIndexer')
embark_encoder = OneHotEncoder(inputCol='EmbarkedIndexer',outputCol='EmbarkedVec')

In [38]:
assembler = VectorAssembler(inputCols=['Pclass','SexVec','Age','SibSp','Parch','Fare','EmbarkedVec'],outputCol='features')

In [39]:
from pyspark.ml import Pipeline

logreg = LogisticRegression(featuresCol='features',labelCol='Survived')

pipeline = Pipeline(stages=[gender_indexer,embark_indexer,gender_encoder,embark_encoder,assembler,logreg])

In [40]:
# to divide into train and test
train_df,test_df = myfinal.randomSplit([0.7,0.3])

In [41]:
df_model = pipeline.fit(train_df)
res_df = df_model.transform(test_df)


In [43]:
df_evalb = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Survived')

# df_evalb(res_df)

res_df.select('Survived','prediction').show()

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
+--------+----------+
only showing top 20 rows



In [45]:
acc = df_evalb.evaluate(res_df)
acc

0.8264885222381637